In [2]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
import math
import random
import numpy as np
from scipy.optimize import minimize

In [3]:
def generate_ansatz(noq, qc, angle):
    depth = noq
    off = 0
    for j in range(depth):
        for i in range(noq):
            qc.ry(angle[off + i], i)
            if (i != 0 and j != (depth - 1)):
                qc.cz(0,i)
        off += i+1

In [13]:
def had_test_b(qc, ancilla_index, gate, noq, angle, b):
    qc.h(ancilla_index)
    generate_ansatz(noq, qc, angle)
    append_A_matrix(qc, gate, ancilla_index)
    append_A_matrix(qc, gate, ancilla_index)
    generate_B(noq, qc, b)
    qc.h(ancilla_index)

In [42]:
def append_A_matrix(qc, gate, ancilla):
    for i in range(len(gate)):
        if (gate[i] == 1):
             qc.cx(ancilla, i)
        if (gate[i] == 2):
            qc.cy(ancilla, i)   
        if (gate[i] == 3):
            qc.cz(ancilla, i)             

In [25]:
def generate_B(noq, qc, b):
    qc.initialize(b, range(noq))

In [45]:
def had_test_a(qc, ancilla_index, gate, gate_trans, noq, angle):
    qc.h(ancilla_index)
    generate_ansatz(noq, qc, angle)
    append_A_matrix(qc, gate, ancilla_index)
    append_A_matrix(qc, gate_trans, ancilla_index)
    qc.h(ancilla_index)

In [29]:
import numpy as np
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
pauli_i = [I, 0]
pauli_x = [X, 1]
pauli_y = [Y, 2]
pauli_z = [Z, 3]

def get_basis_set(num_of_qubit = 1):
    basis_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    int_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    temp_set = []
    for i in range (num_of_qubit-1):
        for j in range (len(basis_set)):
            for k in range (len(int_set)):
                a = basis_set[j]
                b = int_set[k]
                #temp_set.append()
                temp_set.append([np.kron(a[0],b[0]),[a[1],b[1]]])
        int_set = temp_set
        #print(int_set)
        temp_set = []
    return int_set

def get_decomposition(A, num_of_q):
    basis = get_basis_set(num_of_q)
    dec_list = []
    for i in range(len(basis)):
        b = basis[i]
        coeff = np.trace(A*b[0])/(2**num_of_q)
        dec_list.append([coeff, b[1]])
    return dec_list
def get_arbitary_decomposition(A):
    size = A.shape
    size = size[0]
    power = math.log2(size)
    p = int(power)
    if (p != power):
        p += 1
    ret = np.zeros([2**p,2**p], dtype = float)
    ret[:size, :size] = A
    for i in range(size, 2**p):
        ret[i,i] = 1
    return get_decomposition(ret, p)

In [55]:
noq = 2

angle = [np.pi/2, np.pi/8, np.pi/6, np.pi/3]
a = np.matrix('1 2 3 5; 5 4 3 1; 1 3 3 1; 2 3 4 7')
b = [0.7071067, 0.7071067, 0.7071067, 0.7071067]
a = a/np.linalg.norm(b)
b = b/np.linalg.norm(b)
dec_list = get_arbitary_decomposition(a)
gate = []
coeff = []
for i in range(len(dec_list)):
    coeff.append(dec_list[i][0])
    gate.append(dec_list[i][1])

denom = 0
numer = 0
cof = 0
backend = Aer.get_backend('statevector_simulator')
for i in range(len(coeff)):
    for j in range(len(coeff)):
        circ = QuantumCircuit(noq + 1,1)
        cof = coeff[i]*coeff[j]
        had_test_a(circ, noq, gate[i], gate[j], noq, angle)
        circ.measure(noq, 0)
        job = execute(circ, backend)
        result = job.result()
        outputstate = result.get_statevector(circ, decimals=100)
        o = outputstate
        m_sum = 0
        for l in range (0, len(o)):
            if (l%2 == 1):
                n = float(o[l])**2
                m_sum+=n
        denom += cof*m_sum

for i in range(len(coeff)):
    circ = QuantumCircuit(noq + 1,1)
    had_test_b(circ, noq, gate[i], noq, angle, b)
    circ.measure(noq, 0)
    job = execute(circ, backend)
    result = job.result()
    outputstate = result.get_statevector(circ, decimals=100)
    o = outputstate
    m_sum = 0
    for l in range (0, len(o)):
        if (l%2 == 1):
            n = float(o[l])**2
            m_sum+=n
    numer += coeff[i]*m_sum
    
numer *= numer 
print(1-(numer/denom))

c:\users\hrush\.conda\envs\quantum_computing\lib\site-packages\ipykernel_launcher.py:32: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\hrush\.conda\envs\quantum_computing\lib\site-packages\ipykernel_launcher.py:47: ComplexWarning: Casting complex values to real discards the imaginary part


(0.6188735286550548-0.03553539355633342j)


In [59]:

def calculate_cost(angle):
    a = np.matrix('1 2 3 5; 5 4 3 1; 1 3 3 1; 2 3 4 7')
    b = [0.7071067, 0.7071067, 0.7071067, 0.7071067]
    a = a/np.linalg.norm(b)
    b = b/np.linalg.norm(b)
    dec_list = get_arbitary_decomposition(a)
    gate = []
    coeff = []
    for i in range(len(dec_list)):
        coeff.append(dec_list[i][0])
        gate.append(dec_list[i][1])
    
    denom = 0
    numer = 0
    cof = 0
    backend = Aer.get_backend('statevector_simulator')
    for i in range(len(coeff)):
        for j in range(len(coeff)):
            circ = QuantumCircuit(noq + 1,1)
            cof = coeff[i]*coeff[j]
            had_test_a(circ, noq, gate[i], gate[j], noq, angle)
            circ.measure(noq, 0)
            job = execute(circ, backend)
            result = job.result()
            outputstate = result.get_statevector(circ, decimals=100)
            o = outputstate
            m_sum = 0
            for l in range (0, len(o)):
                if (l%2 == 1):
                    n = float(o[l])**2
                    m_sum+=n
            denom += cof*m_sum
    
    for i in range(len(coeff)):
        circ = QuantumCircuit(noq + 1,1)
        had_test_b(circ, noq, gate[i], noq, angle, b)
        circ.measure(noq, 0)
        job = execute(circ, backend)
        result = job.result()
        outputstate = result.get_statevector(circ, decimals=100)
        o = outputstate
        m_sum = 0
        for l in range (0, len(o)):
            if (l%2 == 1):
                n = float(o[l])**2
                m_sum+=n
        numer += coeff[i]*m_sum
        
    numer *= numer 
    return abs(1-(numer/denom))  

In [65]:
angle = [np.pi/2, np.pi/8, np.pi/6, np.pi/3]
maxiter = 2000000
iters = 0 
threshold = 0.00000047
out = minimize(calculate_cost, x0=angle, method="COBYLA", options={'maxiter':2000}, tol = threshold)
print(out)

c:\users\hrush\.conda\envs\quantum_computing\lib\site-packages\ipykernel_launcher.py:30: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\hrush\.conda\envs\quantum_computing\lib\site-packages\ipykernel_launcher.py:45: ComplexWarning: Casting complex values to real discards the imaginary part


     fun: 0.26772549775605975
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 66
  status: 1
 success: True
       x: array([ 0.29938778,  0.12390798,  0.2118103 , -0.38052513])


In [31]:
a = np.matrix('1 2 3 5; 5 4 3 1; 1 3 3 1; 2 3 4 7')
b = np.matrix('0.7071067;0.7071067;0.7071067;0.7071067')
ans = np.linalg.inv(a)*b
print((ans)/np.linalg.norm(ans))

[[ 0.15249857]
 [-0.60999428]
 [ 0.76249285]
 [-0.15249857]]
